<a href="https://colab.research.google.com/github/IamLucifer0312/Furniture-Recognition-CNN/blob/henry-task-3/Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>